In [1]:
# pyspellchecker helps with fixing typos
# !python3 -m pip install pyspellchecker
# !python3 -m spacy download en_core_web_lg


# Sentiment Analysis on Yelp dataset

## Loading libraries


In [2]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import re
import seaborn as sns
import spacy
from matplotlib import rcParams

# import modules to help with preprocessing
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.corpus import stopwords
from nltk.probability import FreqDist

# import modules to help with collecting & counting the tokens
from itertools import chain

from typing import Union

# to trying to fix typos
# from spellchecker import SpellChecker

%matplotlib inline
rcParams["figure.figsize"] = (16, 9)
plt.style.use("ggplot")  # personal preference


## Loading the dataset


In [3]:
IO_TRAIN = "../input/yelp-review-dataset/yelp_review_polarity_csv/train.csv"
# quick checking the file, turns out it is missing the header
ylp = pd.read_csv(IO_TRAIN, header=None)
ylp.columns = ["sentiment", "review"]  # replacing numeric columns with meaningful names
# from readme.txt, newlines are replaced by '\n', which appears in string as escaped '\\n'
ylp["review"] = ylp["review"].apply(lambda rev: re.sub(r"\\n", "\n", rev))
# preview the data
ylp.head()


,sentiment,review
0,1,"Unfortunately, the frustration of being Dr. Go..."
1,2,Been going to Dr. Goldberg for over 10 years. ...
2,1,I don't know what Dr. Goldberg was like before...
3,1,I'm writing this review to give you a heads up...
4,2,All the food is great here. But the best thing...


to be able to deal with classes, it should be in some human interpretable form, so using knowledge from `readme.txt`, class 1 is `NEG` for negative, and class 2 is `POS` for positive


In [4]:
ylp.replace({1: "NEG", 2: "POS"}, inplace=True)
ylp["sentiment"] = ylp["sentiment"].astype("category")


### define globals


In [5]:
REGEX_URL = r"(?:https?://|www\.)\S+"
nlp = spacy.load("en_core_web_lg")
# combining the stopwords from gensim & nltk
STOPWORDS = STOPWORDS.union(frozenset(stopwords.words("english")), nlp.Defaults.stop_words)


## Preprocessing

load libraries to help with preprocessing, & define preprocessing subroutine


In [6]:
# import spacy

# correct = SpellChecker().correctionspacy.lang.en
# nlp = spacy.load("en_core_web_lg")
STOPWORDS = STOPWORDS.union(nlp.Defaults.stop_words)

# to_disable = ["tok2vec", "tagger", "parser", "attribute_ruler", "ner"]
to_disable = ["tok2vec", "parser", "ner"]

# define preprocess subroutine
def preprocess(
    doc: str, stopwords: frozenset = STOPWORDS, **kwargs
) -> spacy.tokens.doc.Doc:
    """
    TODO: fill in pydoc
    """
    # remove any hyperlinks
    doc_processed = re.sub(REGEX_URL, "", doc)

    # takes care of tokenising, lowercasing, and removing punctuations
    doc_processed = simple_preprocess(
        doc_processed,
        min_len=kwargs.get("min_len", 2),
        max_len=kwargs.get("max_len", 15),
    )

    # dropping stopwords & fixing typos (if any)
    doc_processed = [
        # FIXME: correct takes a long time, skip for now
        # correct(token) for token in doc_processed if token not in stopwords
        token
        for token in doc_processed
        if token not in stopwords
    ]

    # lemmatising the tokens
    return nlp(" ".join(doc_processed), disable=to_disable)


def preprocess_generator(
    docs,
    batch_size=10000,
    stopwords: frozenset = STOPWORDS,
    **kwargs
):
    """
    TODO:

    """
    n = len(docs)
    for i in range(0, n, batch_size):
        yield docs.iloc[i : i + batch_size].apply(preprocess, stopwords, **kwargs)


In [7]:
ylp_processed = ylp.copy()  # a clean copy of raw data for preprocessing
reviews = ylp_processed["review"]


In [8]:
result = pd.Series(dtype='object')
for res in preprocess_generator(reviews):
    result = result.append(res)


In [9]:
ylp_processed["review"] = result
# dropping empty reviews
ylp_processed = ylp_processed[ylp_processed["review"].apply(len) > 0]
ylp_processed.shape, ylp.shape


((559907, 2), (560000, 2))

let's compare the processed reviews against raw reviews


In [10]:
ylp.shape[0] - ylp_processed.shape[0]


93

we have dropped $93$ records on account of being empty reviews, after preprocessing


In [11]:
ylp_processed["review"] = ylp_processed["review"].apply(
    lambda doc: " ".join([token.lemma_ for token in doc])
)


In [12]:
ylp_processed.to_csv("train_processed.csv")


---

## Benchmark model

Will start by using a naive Bayes classifier with unigram

naive Bayes is one of the oldest & fastest classifiers for text classifications


In [13]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import cross_validate, train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.metrics import classification_report


In [14]:
X = ylp_processed["review"]
y = ylp_processed["sentiment"]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

benchmark = make_pipeline(TfidfVectorizer(), MultinomialNB())


In [15]:
scores = cross_validate(benchmark, X, y, cv=5, n_jobs=-1, verbose=1)
np.mean(scores["test_score"])


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:  1.6min finished


0.8749453136757473

The benchmark model gives accuracy of $0.875$


In [16]:
_ = benchmark.fit(X_train, y_train)
y_pred = benchmark.predict(X_test)
print(classification_report(y_test, y_pred, digits=4))


              precision    recall  f1-score   support

         NEG     0.8790    0.8778    0.8784     55986
         POS     0.8780    0.8791    0.8786     55996

    accuracy                         0.8785    111982
   macro avg     0.8785    0.8785    0.8785    111982
weighted avg     0.8785    0.8785    0.8785    111982

